In [28]:
import pickle
from commonfunctions import *
import cv2

In [29]:
def HoG(image):
    # Resize the image to 130x276
    resized_img = cv2.resize(image, (64, 128))
    # print(resized_img.shape)

    # Convert the original image to gray scale
    resized_img_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    '''
    # Specify the parameters for our HOG descriptor
    win_size = (64, 128)  # You need to set a proper window size
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    num_bins = 9
    '''
    win_size = (64, 128)  # Increase the window size
    block_size = (32, 32)  # Increase the block size
    block_stride = (16, 16)  # Increase the block stride
    cell_size = (16, 16)  # Increase the cell size
    num_bins = 18  # Increase the number of bins

    # Set the parameters of the HOG descriptor using the variables defined above
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    # Compute the HOG Descriptor for the gray scale image
    hog_descriptor = hog.compute(resized_img_gray)
    return hog_descriptor

def img_pre_processing(image):

    # (1) crop background
    #cropped_image=crop_background(image)

    # (2) Apply Gaussian filtering to smooth out the image and remove any noise that may affect the accuracy of the HOG feature extractor
    #we need to try different kernel sizes for the best!
    gaussian_image = cv2.GaussianBlur(image, (3, 3), 0)

    # Convert the image to floating point
    gaussian_image_float = gaussian_image.astype(np.float32)

    # Calculate the histogram
    hist = cv2.calcHist([image], [0], None, [256], [0,256])

    # Calculate the mean of the histogram
    mean_hist = np.mean(hist)

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Extract the S channel
    s_channel = hsv_image[:, :, 1]

    # Calculate the average saturation
    average_saturation = np.mean(s_channel)

    # Decide the gamma value
    if (average_saturation > 128) and (mean_hist > 128):
        gamma = 1.2  # or any value greater than 1
    else:
        gamma = 0.8  # or any value less than 1

    # Apply gamma correction
    gamma_corrected_image = cv2.pow(gaussian_image_float/ 255, gamma)


    final_image_before_HOG=gamma_corrected_image*255
    
    image=final_image_before_HOG.astype(np.uint8)

    
    return image

In [30]:
#get pre trained model 
with open("Saudi_model.p", "rb") as model_file:
    trained_model = pickle.load(model_file)

In [33]:
img=io.imread("69.jpg")
image=img_pre_processing(img)
feat=HoG(img)
feat2=feat.reshape(1,-1)

In [34]:
classes=['1SAR','5SAR','10SAR','50SAR','100SAR','500SAR']
#make predictions
predictions = trained_model.predict(feat2)
per=trained_model.predict_proba(feat2)
print(per)
#convert predictions variable to int
predictions = int(predictions)
#type(predictions)
print(classes[predictions])

[[0.0050381  0.13947059 0.21004929 0.32206447 0.02607845 0.2972991 ]]
500SAR
